<a href="https://colab.research.google.com/github/heitingv/Masters_project/blob/master/Fine-Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for Fine-Tuning: Modify accordingly witht the necessary pre-trained network and freeze correct layers of current network used

Install pydicom and barbar

In [2]:
!pip install pydicom
!pip install barbar

     |████████████████████████████████| 35.5MB 89kB/s 


Link to drive for data 

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


Import all libraries

In [0]:
import os
import matplotlib
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pydicom
from tqdm import tqdm_notebook as tqdm
from random import randint

import time
from barbar import Bar 
import progressbar
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
from tqdm import trange
from time import sleep
from torch.utils.data.sampler import SubsetRandomSampler 
use_gpu = torch.cuda.is_available()

Class for data only with mass:
returns image/mask padded (for patch extraction) and image/mask resized to 250x250

In [0]:
class Dataset(BaseDataset):

    CLASSES = ['non tumor','tumor']

    def __init__(self, images_dir, masks_dir, classes=None):
        self.ids_f=[]
        self.ids_m_f=[]
        self.ids = os.listdir(images_dir)
        self.ids_m = os.listdir(masks_dir)
        for i in range(len(self.ids)):
          self.ids[i]=self.ids[i].rstrip(".dcm")
          for i in range(len(self.ids_m)):
            self.ids_m[i]=self.ids_m[i].rstrip(".png")

        for temp in self.ids_m:
          if temp in self.ids:
            self.ids_f.append(temp+'.dcm')
            self.ids_m_f.append(temp+'.png')

        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids_f]
        self.masks_fps = [os.path.join(masks_dir, mask_id) for mask_id in self.ids_m_f]
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
    
    def __len__(self):
        return len(self.ids_f)
    
    def breast_left_or_right(self, image_array):
      position=None
      image=(image_array>0).float() #transform image into binary for easier analysis
      coordinates_breast_tissue=(image==1).nonzero() #look at coordinatex where there is '1'
      min_coordinates=torch.min(coordinates_breast_tissue,0)[0][1].item() #find the minimum column of where breast, if breast on the left then 0/1, if right then high number
      
      if  min_coordinates<=100:
        position='left'
      else:
        position='right'
      
      return(position)

    def image_padding(self, position, image_array, mask_array):

      if image_array.shape[0]==4084:
        if position=='left':
          image_tensor=torch.nn.functional.pad(image_array, (0,(3500-3328),0,(4250-4084)))
          mask_tensor=torch.nn.functional.pad(mask_array, (0,(3500-3328),0,(4250-4084)))
        else:
          image_tensor=torch.nn.functional.pad(image_array, ((3500-3328),0,0,(4250-4084)))
          mask_tensor=torch.nn.functional.pad(mask_array, ((3500-3328),0,0,(4250-4084)))
      
      else:
        if position=='left':
          image_tensor=torch.nn.functional.pad(image_array, (0,(2750-2560),0,(3500-3328)))
          mask_tensor=torch.nn.functional.pad(mask_array, (0,(2750-2560),0,(3500-3328)))
        else:
          image_tensor=torch.nn.functional.pad(image_array, ((2750-2560),0,0,(3500-3328)))
          mask_tensor=torch.nn.functional.pad(mask_array, ((2750-2560),0,0,(3500-3328)))
      
      return(image_tensor,mask_tensor)


    def __getitem__(self, i):
        
        # read data
        image = pydicom.dcmread(self.images_fps[i])
        image = image.pixel_array.astype('float')
        image_re = cv2.resize(image,(250,250))
        image = torch.from_numpy(image)
      
        mask = cv2.imread(self.masks_fps[i])
        mask_re = cv2.resize(mask,(250,250))
        mask_re = torch.from_numpy(mask_re)
        mask_re = mask_re.long()
        mask_re = abs((mask_re.sum(2)/3)-1)
        mask_re = (mask_re>0).float()
        mask = torch.from_numpy(mask)
        mask = mask.long()
        mask = abs((mask.sum(2)/3)-1)
        mask = (mask>0).float()

        position=self.breast_left_or_right(image)
        image_pad, mask_pad = self.image_padding(position,image,mask)
        
        return image_pad, mask_pad, image_re, mask_re

Class for data without mass: returns image/mask padded (for patch extraction) and image/mask resized to 250x250

The distinction between the two classes is be able to create a balanced dataset for training

In [0]:
class Dataset_with_NonMass(BaseDataset): #original data non padded or changed in size 

    CLASSES = ['non tumor','tumor']

    def __init__(self, images_dir, masks_dir, classes=None):
        self.ids_f=[]
        self.ids_m_f=[]
        self.ids = os.listdir(images_dir)
        self.ids_m = os.listdir(masks_dir)
        self.images_fps=[]
        self.masks_fps=[]
        for i in range(len(self.ids)):
          self.ids[i]=self.ids[i].rstrip(".dcm")
          for i in range(len(self.ids_m)):
            self.ids_m[i]=self.ids_m[i].rstrip(".png")

        for i in range(len(self.ids)):
          temp=self.ids[i][0:8]
          if temp in self.ids_m:
            self.ids_f.append(self.ids[i]+'.dcm')
            self.ids_m_f.append(temp+'.png')

        for i in range(len(self.ids_m_f)):
          mask_id=self.ids_m_f[i]
          image_id=self.ids_f[i]
          temp=os.path.join(masks_dir, mask_id)
          presence=self.test_mass(temp)

          if presence==False:
            self.images_fps.append(os.path.join(images_dir, image_id))
            self.masks_fps.append(os.path.join(masks_dir, mask_id))

          temp=None
          mask_id=None
          image_id=None

        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

    def test_mass(self,name):
      presence=None

      mask = cv2.imread(name)
      mask = torch.from_numpy(mask)
      mask=mask.long()
      mask=abs((mask.sum(2)/3)-1)
      mask = (mask>0).float()

      if 1 in mask:
        presence=True
      else:
        presence=False

      return(presence)

    def __len__(self):
        return len(self.images_fps)

    def breast_left_or_right(self, image_array):
      position=None
      image=(image_array>0).float() #transform image into binary for easier analysis
      coordinates_breast_tissue=(image==1).nonzero() #look at coordinatex where there is '1'
      min_coordinates=torch.min(coordinates_breast_tissue,0)[0][1].item() #find the minimum column of where breast, if breast on the left then 0/1, if right then high number
      
      if  min_coordinates<=100:
        position='left'
      else:
        position='right'
      
      return(position)

    def image_padding(self, position, image_array, mask_array):

      if image_array.shape[0]==4084:
        if position=='left':
          image_tensor=torch.nn.functional.pad(image_array, (0,(3500-3328),0,(4250-4084)))
          mask_tensor=torch.nn.functional.pad(mask_array, (0,(3500-3328),0,(4250-4084)))
        else:
          image_tensor=torch.nn.functional.pad(image_array, ((3500-3328),0,0,(4250-4084)))
          mask_tensor=torch.nn.functional.pad(mask_array, ((3500-3328),0,0,(4250-4084)))
      
      else:
        if position=='left':
          image_tensor=torch.nn.functional.pad(image_array, (0,(2750-2560),0,(3500-3328)))
          mask_tensor=torch.nn.functional.pad(mask_array, (0,(2750-2560),0,(3500-3328)))
        else:
          image_tensor=torch.nn.functional.pad(image_array, ((2750-2560),0,0,(3500-3328)))
          mask_tensor=torch.nn.functional.pad(mask_array, ((2750-2560),0,0,(3500-3328)))
      
      return(image_tensor,mask_tensor)

    def __getitem__(self, i):
        
        # read data
        image = pydicom.dcmread(self.images_fps[i])
        image = image.pixel_array.astype('float')
        image_re = cv2.resize(image,(250,250))
        image = torch.from_numpy(image)
      
        mask = cv2.imread(self.masks_fps[i])
        mask_re = cv2.resize(mask,(250,250))
        mask_re = torch.from_numpy(mask_re)
        mask_re = mask_re.long()
        mask_re = abs((mask_re.sum(2)/3)-1)
        mask_re = (mask_re>0).float()
        mask = torch.from_numpy(mask)
        mask = mask.long()
        mask = abs((mask.sum(2)/3)-1)
        mask = (mask>0).float()

        position=self.breast_left_or_right(image)
        image_pad, mask_pad = self.image_padding(position,image,mask)

        return image_pad, mask_pad, image_re, mask_re

Load train, validation and test dataset from drive that was previously saved so that all trainings/testings are done on exactly the same images

In [0]:
test_loader=(torch.load('/gdrive/My Drive/test_loader.pth'))
train_loader=(torch.load('/gdrive/My Drive/train_loader.pth'))
val_loader=(torch.load('/gdrive/My Drive/val_loader.pth'))

Interchange between UNet, UNet++ and FC-DenseNet depending
Here the standard UNet is used as example with its contracting path frozen
Further down in the code, I will add in the same thing for UNet++ and FC-DenseNet for additional information

In [0]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
      
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

Model

In [5]:
model = UNet(n_classes=2, n_channels=1)
for name, child in model.named_children():
    print(name)

inc
down1
down2
down3
down4
up1
up2
up3
up4
outc


In [10]:
for name, child in model.named_children():
   if name in ['inc','down1','down2','down3']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False

inc is unfrozen
down1 is unfrozen
down2 is unfrozen
down3 is unfrozen
down4 is frozen
up1 is frozen
up2 is frozen
up3 is frozen
up4 is frozen
outc is frozen


In [11]:
!pip install pytorch-model-summary
from pytorch_model_summary import summary
print(summary(model, torch.zeros((1, 1, 250, 250)), show_input=False))

--------------------------------------------------------------------------
      Layer (type)           Output Shape         Param #     Tr. Param #
      DoubleConv-1      [1, 64, 250, 250]          37,824          37,824
            Down-2     [1, 128, 125, 125]         221,952         221,952
            Down-3       [1, 256, 62, 62]         886,272         886,272
            Down-4       [1, 512, 31, 31]       3,542,016       3,542,016
            Down-5       [1, 512, 15, 15]       4,721,664               0
              Up-6       [1, 256, 31, 31]       2,950,656               0
              Up-7       [1, 128, 62, 62]         738,048               0
              Up-8      [1, 64, 125, 125]         184,704               0
              Up-9      [1, 64, 250, 250]         110,976               0
        OutConv-10       [1, 2, 250, 250]             130               0
Total params: 13,394,242
Trainable params: 4,688,064
Non-trainable params: 8,706,178
--------------------------

Function to calculate IoU, Dice coeff etc

In [0]:
def metrics(predicted,truth,word):
  ####### metrics for tumour
  TP=0
  FN=0
  FP=0
  TN=0
  for i in range(truth.squeeze().size()[0]):
    for j in range(truth.squeeze().size()[1]):
      if truth[i,j]==1 and predicted[i,j]==1:
        TP+=1
      elif truth[i,j]==1 and predicted[i,j]==0:
        FN+=1
      elif truth[i,j]==0 and predicted[i,j]==1:
        FP+=1
      else:
        TN+=1
  
  if TP==0 and FP==0 and FN==0:
    iou_tumour=0
    dice_tumour=0
  else:
    iou_tumour = TP/(TP+FP+FN)
    dice_tumour = (2*TP)/(TP+FP+TP+FN)

  if TN==0 and FP==0:
    spec_tumour=0
  else:
    spec_tumour = TN/(TN+FP) #specificity #true negative rate
 
  if TP==0 and FN==0:
    sens_tumour=0
  else:
    sens_tumour = TP/(TP+FN) #sensitivity #true positive rate

  acc_tumour = (TP+TN)/(TP+TN+FP+FN) #accuracy

  ####### metrics for background
  truth_b=abs(truth-1)
  predicted_b=abs(predicted-1)
  TP_b=0
  FN_b=0
  FP_b=0
  TN_b=0
  for i in range(truth_b.squeeze().size()[0]):
    for j in range(truth_b.squeeze().size()[1]):
      if truth_b[i,j]==1 and predicted_b[i,j]==1:
        TP_b+=1
      elif truth_b[i,j]==1 and predicted_b[i,j]==0:
        FN_b+=1
      elif truth_b[i,j]==0 and predicted_b[i,j]==1:
        FP_b+=1
      else:
        TN_b+=1
  
  if TP_b==0 and FP_b==0 and FN_b==0:
    iou_background=0
    dice_background=0
  else:
    iou_background = TP_b/(TP_b+FP_b+FN_b)
    dice_background = (2*TP_b)/(TP_b+FP_b+TP_b+FN_b)

  if TN_b==0 and FP_b==0:
    spec_background=0
  else:
   spec_background = TN_b/(TN_b+FP_b) #specificity #true negative rate
 
  if TP_b==0 and FN_b==0:
    sens_background=0
  else:
    sens_background = TP_b/(TP_b+FN_b) #sensitivity #true positive rate

  acc_background = (TP_b+TN_b)/(TP_b+TN_b+FP_b+FN_b) #accuracy

  ####### metrics for mean of tumour & background
  object_nb=0
  if 1 not in truth:
    object_nb=1
  else:
    object_nb=2
  
  mean_iou=(iou_tumour+iou_background)/object_nb
  mean_dice=(dice_tumour+dice_background)/object_nb
  mean_spec=(spec_tumour+spec_background)/object_nb
  mean_sens=(sens_tumour+sens_background)/object_nb
  mean_acc=(acc_tumour+acc_background)/2
  
  if word=='iou':
    return(iou_tumour,iou_background,mean_iou)
  elif word=='AllTumour':
    return(iou_tumour,dice_tumour,spec_tumour,sens_tumour,acc_tumour)
  elif word=='AllBackground':
    return(iou_background,dice_background,spec_background,sens_background,acc_background)
  elif word=='AllMean':
    return(mean_iou,mean_dice,mean_spec,mean_sens,mean_acc)


Optimizer & criterion: the learning rate need to be multiplied by 10^-1 for fine-tuning

In [0]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

Add the opath to the correct pre-trained model

In [0]:
path_transfer='/networks/FINETUNE_UNET/UNet_WholeMass/'
name_transfer='UNet_WholeMass'
model.load_state_dict(torch.load('/gdrive/My Drive'+path_transfer+name_transfer+'.pth'))

<All keys matched successfully>

In [0]:
model = UNet(n_classes=2, n_channels=1)
path='/networks/FINETUNE_UNET/UNet_WholeMass_finetune/'
name='UNet_WholeMass_finetune'

Training

In [0]:
epochs=20
running_loss = 0.0
best=0.0
best_tumour=0.0
train_iou = 0.0
val_iou = 0.0
val_tumour_iou=[]
counter=0
iou_list=[]
val_iou_list=[]
nb_patches_analyzed=0
nb_patches_analyzed_v=0

file = open('/gdrive/My Drive'+path+'training2.txt','w')
file.write('Training: \n')

time.sleep(5)
for epoch in range(epochs):  

    ###################### Training
    time.sleep(5)
    Bar = progressbar.ProgressBar(max_value=len(train_loader))
    for i, data in enumerate(Bar(train_loader), 0):
        image, mask, image_re, mask_re = data

        optimizer.zero_grad()
        outputs = model(image_re.unsqueeze(dim=0).float()) 
        outputs_final_probabilities, outputs_final = torch.max(outputs.squeeze(), axis=0) 
        loss = criterion(outputs.float(), mask_re.long())
        loss.backward() #backward propagation
        optimizer.step() #optimize
        iou_tumour,iou_background,mean_iou = metrics(outputs_final,mask_re.squeeze(),'iou')
        train_iou+=mean_iou

    ############### Validation
    time.sleep(5)
    BarTwo = progressbar.ProgressBar(max_value=len(val_loader))   
    for i, data_v in enumerate(BarTwo(val_loader), 0):
        image_v, mask_v, image_re_v, mask_re_v = data_v
        outputs_v=model(image_re_v.unsqueeze(dim=0).float())
        outputs_final_probabilities_v, outputs_final_v = torch.max(outputs_v.squeeze(), axis=0)
        iou_tumour,iou_background,mean_iou = metrics(outputs_final_v,mask_re_v.squeeze(),'iou')
        val_iou+=mean_iou
        if 1 in mask_re_v:
          val_tumour_iou.append(iou_tumour)


    ################ save best model
    if val_iou > best:
      best=val_iou
      torch.save(model.state_dict(),'/gdrive/My Drive/'+path+name+'.pth')
    
    if sum(val_tumour_iou) > best_tumour:
      best_tumour=sum(val_tumour_iou)
      torch.save(model.state_dict(),'/gdrive/My Drive/'+path+name+'_tumour.pth')
      
    ################ print for each epoch iou
    print('Epoch %d :iou train: %.3f' % (epoch + 1, train_iou/ len(train_loader)),'; iou val: %.3f' % (val_iou/ len(val_loader)),'; tumour iou val: %.3f' % (sum(val_tumour_iou)/ len(val_tumour_iou)),'\n')
    file.write('\n\nEpoch %d : mean iou train: %.3f' % (epoch + 1, train_iou/ len(train_loader)))
    file.write(' ;mean iou validation: %.3f' % (val_iou/ len(val_loader)))  
    file.write(' ;tumour iou validation: %.3f' % (sum(val_tumour_iou)/ len(val_tumour_iou)))    

    iou_list.append(train_iou/ len(train_loader))
    val_iou_list.append(val_iou/ len(val_loader))

    #print for every 20 epoch
    if (counter%1)==0:
      fig = plt.figure()
      plt.subplot(1, 3, 1)
      plt.imshow(image_re.squeeze())
      plt.subplot(1, 3, 2)
      plt.imshow(mask_re.squeeze())
      plt.subplot(1, 3, 3)
      plt.imshow(outputs_final.detach().numpy())

    counter+=1
    running_loss = 0.0
    train_iou = 0.0
    val_tumour_iou = []
    nb_patches_analyzed=0
    nb_patches_analyzed_v=0
    val_iou=0.0

print('Finished Training')

# file.write('\n\nTrain iou list')
# file.wirte(iou_list)
# file.write('Validation iou_list')
# file.write(val_iou_list)
file.close()


Testing

In [0]:
model.load_state_dict(torch.load('/gdrive/My Drive/'+path+name+'.pth'))

file = open('/gdrive/My Drive'+path+'testing.txt','w')
file.write('Testing \n')

iou_tumour_list=[] 
ground=[] 
test_iou = 0.0
test_dice = 0.0
test_spec = 0.0
test_sens = 0.0
test_acc = 0.0
test_iou_tumour = 0.0
test_dice_tumour = 0.0
test_spec_tumour = 0.0
test_sens_tumour = 0.0
test_acc_tumour = 0.0

false_positive=0
true_positive=0
false_negative=0
true_negative=0
FPR=0
TPR=0
FNR=0
TNR=0
tum_count=0

################ test model
for i, data_t in enumerate(test_loader, 0):
  image_t, mask_t, image_re_t, mask_re_t = data_t
  outputs_t=model(image_re_t.unsqueeze(dim=0).float())
  outputs_final_probabilities_t, outputs_final_t = torch.max(outputs_t.squeeze(), axis=0)
  iou_tumour,iou_background,mean_iou = metrics(outputs_final_t,mask_re_t.squeeze(),'iou')
  test_iou += mean_iou 
  mean_iou,mean_dice,mean_spec,mean_sens,mean_acc= metrics(outputs_final_t,mask_re_t.squeeze(),'AllMean')
  test_dice += mean_dice
  test_spec += mean_spec
  test_sens += mean_sens
  test_acc += mean_acc
  iou_tumour_list.append(iou_tumour)
  print('\n\nTest Image %d : Mean iou %.3f' % (i+1, mean_iou))
  file.write('\n\nTest image %d : Mean iou %.3f, mean dice %.3f, mean spec %.3f, mean sens %.3f, mean acc %.3f' % (i+1,mean_iou,mean_dice,mean_spec,mean_sens,mean_acc))

  if 1 in mask_re_t:
    print('Tumour Iou: %.3f' % (iou_tumour))
    iou_tumour,dice_tumour,spec_tumour,sens_tumour,acc_tumour = metrics(outputs_final_t,mask_re_t.squeeze(),'AllTumour')
    test_iou_tumour += iou_tumour
    test_dice_tumour += dice_tumour
    test_spec_tumour += spec_tumour
    test_sens_tumour += sens_tumour
    test_acc_tumour += acc_tumour
    tum_count+=1
    file.write('\n tumour iou %.3f, tumour dice %.3f, tumour spec %.3f, tumour sens %.3f, tumour acc %.3f' % (iou_tumour,dice_tumour,spec_tumour,sens_tumour,acc_tumour))

  if 1 in mask_re_t:
    ground.append(1)
  else:
    ground.append(0)

  if 1 in mask_re_t.squeeze() and 1 in outputs_final_t.squeeze():
    true_positive=true_positive+1
  elif 1 not in mask_re_t.squeeze() and 1 in outputs_final_t.squeeze():
    false_positive=false_positive+1
  elif 1 in mask_re_t.squeeze() and 1 not in outputs_final_t.squeeze():
    false_negative=false_negative+1
  else:
    true_negative=true_negative+1

  fig = plt.figure()
  plt.subplot(1, 3, 1)
  plt.imshow(image_re_t.squeeze())
  plt.subplot(1, 3, 2)
  plt.imshow(mask_re_t.squeeze())
  plt.subplot(1, 3, 3)
  plt.imshow(outputs_final_t.detach().numpy())
  matplotlib.image.imsave('/gdrive/My Drive'+path+'test2/'+name+'_image_'+str(i)+'.png',image_t.squeeze())
  matplotlib.image.imsave('/gdrive/My Drive'+path+'test2/'+name+'_mask_'+str(i)+'.png',mask_t.squeeze())
  matplotlib.image.imsave('/gdrive/My Drive'+path+'test2/'+name+'_prediction_'+str(i)+'.png',outputs_final_t.detach().numpy())


#print('\nOverall Test Images: test loss: %.3f' % (test_loss / len(test_loader)))
print('\n\n\nOver entire Test data set: average mean iou: %.3f' % (test_iou/ len(test_loader)))
file.write('\n\n\n\nOverall iou %.3f, overall dice %.3f, overall spec %.3f, overall sens %.3f, overall acc %.3f' % (test_iou/len(test_loader),test_dice/len(test_loader),test_spec/len(test_loader),test_sens/len(test_loader),test_acc/len(test_loader)))
print('\nAverage tumour iou (for mask that have tumour): %.3f' % (test_iou_tumour/tum_count), '\n')
file.write('\n\nOverall tumour iou %.3f, overall tumour dice %.3f, overall tumour spec %.3f, overall tumour sens %.3f, tumour overall acc %.3f' % (test_iou_tumour/tum_count,test_dice_tumour/tum_count,test_spec_tumour/tum_count,test_sens_tumour/tum_count,test_acc_tumour/tum_count))

####### from testing
FPR=false_positive/(false_positive+true_negative) #False Positive Rate 
TPR=true_positive/(true_positive+false_negative) #True Positive Rate ###Sens
FNR=false_negative/(false_negative+true_positive) #False Negative Rate
TNR=true_negative/(true_negative+false_positive) #True Negative Rate ###Spec

Acc=(true_negative+true_positive)/(false_positive+false_negative+true_negative+true_positive)

auc=ROC(iou_tumour_list,ground)

# f1, auprc = Recall_Precision(iou_tumour_list,ground)

# file.write('\n\n-------------------')
# file.write('\nAUC: ')
# file.write(auc)
# file.write('\nF1: ',f1,' ;AUPRC: ',auprc)
file.close()

# print('False Positive Rate:', FPR, '\n')
# print('True Positive Rate:', TPR, '\n')
# print('False Negative Rate:', FNR, '\n')
# print('True Negative Rate:', TNR, '\n')
# print('Acc:', acc, '\n')
# print('AUC:', auc, '\n')
# print('F1:', f1, 'AUPRC:', auprc, '\n')


------------------------------------

UNet++ finetuning freezing layers

In [0]:
class conv_block_nested(nn.Module):

    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)

        x = self.conv2(x)
        x = self.bn2(x)
        output = self.activation(x)

        return output

class Nested_UNet(nn.Module):

    def __init__(self, in_ch=1, out_ch=2):
        super(Nested_UNet, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)

    def resize(self,x1,x2):
      x1 = self.Up(x1)
      diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
      diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

      x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                      diffY // 2, diffY - diffY // 2])

      return x1

    def forward(self, x):

        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.resize(x1_0,x0_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.resize(x2_0,x1_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.resize(x1_1,x0_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.resize(x3_0,x2_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.resize(x2_1,x1_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.resize(x1_2,x0_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.resize(x4_0,x3_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.resize(x3_1,x2_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.resize(x2_2,x1_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.resize(x1_3,x0_3)], 1))

        output = self.final(x0_4)
        return output

In [22]:
model=Nested_UNet()
for name, child in model.named_children():
    print(name)

pool
Up
conv0_0
conv1_0
conv2_0
conv3_0
conv4_0
conv0_1
conv1_1
conv2_1
conv3_1
conv0_2
conv1_2
conv2_2
conv0_3
conv1_3
conv0_4
final


In [23]:
for name, child in model.named_children():
   if name in ['pool','Up','conv0_0','conv1_0','conv2_0','conv3_0','conv0_1','conv1_1','conv2_1','conv0_2']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False

pool is unfrozen
Up is unfrozen
conv0_0 is unfrozen
conv1_0 is unfrozen
conv2_0 is unfrozen
conv3_0 is unfrozen
conv4_0 is frozen
conv0_1 is unfrozen
conv1_1 is unfrozen
conv2_1 is unfrozen
conv3_1 is frozen
conv0_2 is unfrozen
conv1_2 is frozen
conv2_2 is frozen
conv0_3 is frozen
conv1_3 is frozen
conv0_4 is frozen
final is frozen


In [24]:
!pip install pytorch-model-summary
from pytorch_model_summary import summary
print(summary(model, torch.zeros((1, 1, 28, 28)), show_input=False))

-----------------------------------------------------------------------------
           Layer (type)         Output Shape         Param #     Tr. Param #
    conv_block_nested-1      [1, 64, 28, 28]          37,824          37,824
            MaxPool2d-2      [1, 64, 14, 14]               0               0
    conv_block_nested-3     [1, 128, 14, 14]         221,952         221,952
             Upsample-4     [1, 128, 28, 28]               0               0
    conv_block_nested-5      [1, 64, 28, 28]         147,840         147,840
            MaxPool2d-6       [1, 128, 7, 7]               0               0
    conv_block_nested-7       [1, 256, 7, 7]         886,272         886,272
             Upsample-8     [1, 256, 14, 14]               0               0
    conv_block_nested-9     [1, 128, 14, 14]         590,592         590,592
            Upsample-10     [1, 128, 28, 28]               0               0
   conv_block_nested-11      [1, 64, 28, 28]         184,704         184,70

----------------------------------------------------------------------------

FC DenseNet finetuning freezing layers

In [0]:
class DenseLayer(nn.Sequential):
    def __init__(self, in_channels, growth_rate):
        super().__init__()
        self.add_module('norm', nn.BatchNorm2d(in_channels))
        self.add_module('relu', nn.ReLU(True))
        self.add_module('conv', nn.Conv2d(in_channels, growth_rate, kernel_size=3,
                                          stride=1, padding=1, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))

    def forward(self, x):
        return super().forward(x)


class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers, upsample=False):
        super().__init__()
        self.upsample = upsample
        self.layers = nn.ModuleList([DenseLayer(
            in_channels + i*growth_rate, growth_rate)
            for i in range(n_layers)])

    def forward(self, x):
        if self.upsample:
            new_features = []
            #we pass all previous activations into each dense layer normally
            #But we only store each dense layer's output in the new_features array
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1)
                new_features.append(out)
            return torch.cat(new_features,1)
        else:
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1) # 1 = channel axis
            return x


class TransitionDown(nn.Sequential):
    def __init__(self, in_channels):
        super().__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(in_channels, in_channels,
                                          kernel_size=1, stride=1,
                                          padding=0, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))
        self.add_module('maxpool', nn.MaxPool2d(2))

    def forward(self, x):
        return super().forward(x)


class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convTrans = nn.ConvTranspose2d(
            in_channels=in_channels, out_channels=out_channels,
            kernel_size=3, stride=2, padding=0, bias=True)

    def forward(self, x, skip):
        out = self.convTrans(x)
        out = center_crop(out, skip.size(2), skip.size(3))
        out = torch.cat([out, skip], 1)
        return out


class Bottleneck(nn.Sequential):
    def __init__(self, in_channels, growth_rate, n_layers):
        super().__init__()
        self.add_module('bottleneck', DenseBlock(
            in_channels, growth_rate, n_layers, upsample=True))

    def forward(self, x):
        return super().forward(x)


def center_crop(layer, max_height, max_width):
    _, _, h, w = layer.size()
    xy1 = (w - max_width) // 2
    xy2 = (h - max_height) // 2
    return layer[:, :, xy2:(xy2 + max_height), xy1:(xy1 + max_width)]


class FCDenseNet(nn.Module):
    def __init__(self, in_channels=1, down_blocks=(5,5,5,5,5),
                 up_blocks=(5,5,5,5,5), bottleneck_layers=5,
                 growth_rate=16, out_chans_first_conv=48, n_classes=2):
        super().__init__()
        self.down_blocks = down_blocks
        self.up_blocks = up_blocks
        cur_channels_count = 0
        skip_connection_channel_counts = []

        ## First Convolution ##

        self.add_module('firstconv', nn.Conv2d(in_channels=in_channels,
                  out_channels=out_chans_first_conv, kernel_size=3,
                  stride=1, padding=1, bias=True))
        cur_channels_count = out_chans_first_conv

        #####################
        # Downsampling path #
        #####################

        self.denseBlocksDown = nn.ModuleList([])
        self.transDownBlocks = nn.ModuleList([])
        for i in range(len(down_blocks)):
            self.denseBlocksDown.append(
                DenseBlock(cur_channels_count, growth_rate, down_blocks[i]))
            cur_channels_count += (growth_rate*down_blocks[i])
            skip_connection_channel_counts.insert(0,cur_channels_count)
            self.transDownBlocks.append(TransitionDown(cur_channels_count))

        #####################
        #     Bottleneck    #
        #####################

        self.add_module('bottleneck',Bottleneck(cur_channels_count,
                                     growth_rate, bottleneck_layers))
        prev_block_channels = growth_rate*bottleneck_layers
        cur_channels_count += prev_block_channels

        #######################
        #   Upsampling path   #
        #######################

        self.transUpBlocks = nn.ModuleList([])
        self.denseBlocksUp = nn.ModuleList([])
        for i in range(len(up_blocks)-1):
            self.transUpBlocks.append(TransitionUp(prev_block_channels, prev_block_channels))
            cur_channels_count = prev_block_channels + skip_connection_channel_counts[i]

            self.denseBlocksUp.append(DenseBlock(
                cur_channels_count, growth_rate, up_blocks[i],
                    upsample=True))
            prev_block_channels = growth_rate*up_blocks[i]
            cur_channels_count += prev_block_channels

        ## Final DenseBlock ##

        self.transUpBlocks.append(TransitionUp(
            prev_block_channels, prev_block_channels))
        cur_channels_count = prev_block_channels + skip_connection_channel_counts[-1]

        self.denseBlocksUp.append(DenseBlock(
            cur_channels_count, growth_rate, up_blocks[-1],
                upsample=False))
        cur_channels_count += growth_rate*up_blocks[-1]

        ## Softmax ##

        self.finalConv = nn.Conv2d(in_channels=cur_channels_count,
               out_channels=n_classes, kernel_size=1, stride=1,
                   padding=0, bias=True)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out = self.firstconv(x)

        skip_connections = []
        for i in range(len(self.down_blocks)):
            out = self.denseBlocksDown[i](out)
            skip_connections.append(out)
            out = self.transDownBlocks[i](out)

        out = self.bottleneck(out)
        for i in range(len(self.up_blocks)):
            skip = skip_connections.pop()
            out = self.transUpBlocks[i](out, skip)
            out = self.denseBlocksUp[i](out)

        out = self.finalConv(out)
        out = self.softmax(out)
        return out


def FCDenseNet57(n_classes):
    return FCDenseNet(
        in_channels=1, down_blocks=(4, 4, 4, 4, 4),
        up_blocks=(4, 4, 4, 4, 4), bottleneck_layers=4,
        growth_rate=12, out_chans_first_conv=48, n_classes=n_classes)


def FCDenseNet67(n_classes):
    return FCDenseNet(
        in_channels=1, down_blocks=(5, 5, 5, 5, 5),
        up_blocks=(5, 5, 5, 5, 5), bottleneck_layers=5,
        growth_rate=16, out_chans_first_conv=48, n_classes=n_classes)


def FCDenseNet103(n_classes):
    return FCDenseNet(
        in_channels=1, down_blocks=(4,5,7,10,12),
        up_blocks=(12,10,7,5,4), bottleneck_layers=15,
        growth_rate=16, out_chans_first_conv=48, n_classes=n_classes)

In [26]:
model = FCDenseNet103(n_classes=2)
for name, child in model.named_children():
    print(name)

firstconv
denseBlocksDown
transDownBlocks
bottleneck
transUpBlocks
denseBlocksUp
finalConv
softmax


In [27]:
for name, child in model.named_children():
   if name in ['softmax','firstconv','denseBlocksDown','transDownBlocks']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False

firstconv is unfrozen
denseBlocksDown is unfrozen
transDownBlocks is unfrozen
bottleneck is frozen
transUpBlocks is frozen
denseBlocksUp is frozen
finalConv is frozen
softmax is unfrozen


In [28]:
!pip install pytorch-model-summary
from pytorch_model_summary import summary
print(summary(model, torch.zeros((1, 1, 250, 250)), show_input=False))

--------------------------------------------------------------------------
      Layer (type)           Output Shape         Param #     Tr. Param #
          Conv2d-1      [1, 48, 250, 250]             480             480
      DenseBlock-2     [1, 112, 250, 250]          42,112          42,112
     BatchNorm2d-3     [1, 112, 250, 250]             224             224
            ReLU-4     [1, 112, 250, 250]               0               0
          Conv2d-5     [1, 112, 250, 250]          12,656          12,656
       Dropout2d-6     [1, 112, 250, 250]               0               0
       MaxPool2d-7     [1, 112, 125, 125]               0               0
      DenseBlock-8     [1, 192, 125, 125]         105,200         105,200
     BatchNorm2d-9     [1, 192, 125, 125]             384             384
           ReLU-10     [1, 192, 125, 125]               0               0
         Conv2d-11     [1, 192, 125, 125]          37,056          37,056
      Dropout2d-12     [1, 192, 125, 